<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/cdvae_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inverse Design of Next-Generation Superconductors Using Data-Driven Deep Generative Models

Reference: https://pubs.acs.org/doi/10.1021/acs.jpclett.3c01260

Author: Kamal Choudhary (kamal.choudhary@nist.gov)

In [1]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [2]:
# !conda --version

In [3]:
# import condacolab
# condacolab.check()

In [4]:
%%time
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pytorch-lightning wandb torchmetrics==0.6.0 pymatgen==2022.4.26
!pip install -q hydra-core jarvis-tools python-dotenv p-tqdm accelerate
!pip install -q numpy==1.26.4

2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 36.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Requested pytorch-lightning from https://files.pythonhosted.org/packages/68/73/54dcf2ab4156b9e6db9b050d9806fe49768307457d13f99963db5f227526/pytorch_lightning-1.8.3.post2-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    torch (>=1.9.*)
           ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested pytorch-lightning from https://files.pythonhosted.org/packages/b5/ea/7cb6830785a844eddf45f88b9b14cbf8c3ecce718d4d

In [6]:
# !pip install -q pytorch-lightning==1.3.6

In [5]:
import os
os.chdir('/content')
if not os.path.exists('cdvae'):
  !git clone https://github.com/JARVIS-Materials-Design/cdvae.git

os.chdir('cdvae')
!pip install -e .
!pip install -q numpy==1.26.4

Cloning into 'cdvae'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 253 (delta 47), reused 38 (delta 38), pack-reused 162 (from 1)
Receiving objects: 100% (253/253), 146.76 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Updating files: 100% (99/99), done.
Obtaining file:///content/cdvae
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cdvae


In [6]:
import os
os.chdir('/content/cdvae')
if not os.path.exists("/content/cdvae/WABDB"):
  os.makedirs("/content/cdvae/WABDB")
if not os.path.exists("/content/cdvae/HYDRA_JOBS"):
  os.makedirs("/content/cdvae/HYDRA_JOBS")
os.environ["PROJECT_ROOT"]="/content/cdvae"
os.environ["WABDB"]="/content/cdvae/WABDB"
os.environ["WABDB_DIR"]="/content/cdvae/WABDB"
os.environ["HYDRA_JOBS"]="/content/cdvae/HYDRA_JOBS"
%env HYDRA_FULL_ERROR=1


env: HYDRA_FULL_ERROR=1


In [7]:
!echo $HYDRA_FULL_ERROR

1


Data was generated using this [script](https://github.com/JARVIS-Materials-Design/cdvae/blob/main/scripts/generate_data_cdvae.py).

In [8]:

import yaml
import pprint

with open('/content/cdvae/conf/train/default.yaml','r') as f:
  yam = yaml.safe_load(f)
#yam.pop('early_stopping')
#yam['pl_trainer']['fast_dev_run']=True
yam['pl_trainer']['gpus']=0

with open('/content/cdvae/conf/train/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/optim/default.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['use_lr_scheduler']=False

with open('/content/cdvae/conf/optim/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

{'deterministic': False,
 'early_stopping': {'patience': '${data.early_stopping_patience}',
                    'verbose': False},
 'model_checkpoints': {'save_top_k': 1, 'verbose': False},
 'monitor_metric': 'val_loss',
 'monitor_metric_mode': 'min',
 'pl_trainer': {'accumulate_grad_batches': 1,
                'fast_dev_run': False,
                'gpus': 0,
                'gradient_clip_algorithm': 'value',
                'gradient_clip_val': 0.5,
                'max_epochs': '${data.train_max_epochs}',
                'num_sanity_val_steps': 2,
                'precision': 32,
                'profiler': 'simple'},
 'random_seed': 42}
{'lr_scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
                  'factor': 0.6,
                  'min_lr': '1e-4',
                  'patience': 30},
 'optimizer': {'_target_': 'torch.optim.Adam',
               'betas': [0.9, 0.999],
               'eps': '1e-08',
               'lr': 0.001,
               'weight_de

In [4]:
# !rm -rf "/usr/local/lib/python3.11/dist-packages/numpy*"

In [9]:
# !pip cache purge
# !pip uninstall numpy -y
# !pip install numpy==1.26.4
# !python -c "import numpy as np; print(np.__version__)"

In [11]:
!which python
!which pip


/usr/local/bin/python
/usr/local/bin/pip


In [12]:
# !pip uninstall numpy -y
# !pip install numpy==1.26.4 --force-reinstall


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


In [2]:
import sys
print(sys.executable)


/usr/bin/python3.real


In [1]:
import numpy as np
print(np.__version__)

2.0.2


In [ ]:
%%time
import os
os.environ["WANDB_ANONYMOUS"] = "must"
!python cdvae/run.py data=supercon expname=supercon_test02 model.predict_property=True

2025-05-21 22:39:22.503021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747867162.753349    1600 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747867162.823275    1600 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 22:39:23.358331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/content/cdvae/cdvae/run.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability ve

In [13]:
!pip install smact

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 56.2 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16676 sha256=b78a4d9a604279f9c5be58422005b6728c3f366cd92e95f347b5fe0396bc5bcb
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=e7ed82b18219f4e995fa588f8c2b25a65a7ff22dde4d75108b7c725b2000518d
  Stored in directory: /root/.cache/pip/wheels/16/fb/76/306387739cf9d53b1c39b0c8aadbbb17dc05f256756d8fd915
Successfully built pandarallel bibtexparser
  Attempting uninstall: pymatgen
    Found existing installation: pymatgen 2022.4.26
    Uninstalling pymat

Export conda environment

In [14]:
!conda env export

name: base
channels:
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=2_gnu
  - archspec=0.2.3=pyhd8ed1ab_0
  - boltons=24.0.0=pyhd8ed1ab_1
  - brotli-python=1.1.0=py311hfdbb021_2
  - bzip2=1.0.8=h4bc722e_7
  - c-ares=1.34.4=hb9d3cd8_0
  - ca-certificates=2024.12.14=hbcca054_0
  - certifi=2024.12.14=pyhd8ed1ab_0
  - cffi=1.17.1=py311hf29c0ef_0
  - charset-normalizer=3.4.1=pyhd8ed1ab_0
  - colorama=0.4.6=pyhd8ed1ab_1
  - conda=24.11.2=py311h38be061_1
  - conda-libmamba-solver=24.9.0=pyhd8ed1ab_0
  - conda-package-handling=2.4.0=pyh7900ff3_2
  - conda-package-streaming=0.11.0=pyhd8ed1ab_0
  - distro=1.9.0=pyhd8ed1ab_1
  - fmt=11.0.2=h434a139_0
  - frozendict=2.4.6=py311h9ecbd09_0
  - h2=4.1.0=pyhd8ed1ab_1
  - hpack=4.0.0=pyhd8ed1ab_1
  - hyperframe=6.0.1=pyhd8ed1ab_1
  - idna=3.10=pyhd8ed1ab_1
  - jsonpatch=1.33=pyhd8ed1ab_1
  - jsonpointer=3.0.0=py311h38be061_1
  - keyutils=1.6.1=h166bdaf_0
  - krb5=1.21.3=h659f571_0
  - ld_impl_linux-64=2.43=h712a8e

In [15]:
# from datetime import date
# d1 = today.strftime("%Y-%m-%d")

In [16]:
temp_dir=!ls /content/cdvae/HYDRA_JOBS/singlerun/

In [17]:
os.environ['TMP_DIR']=temp_dir[0]

In [18]:
!echo $TMP_DIR

2025-05-21


Adjust path accordingly

In [19]:
%%time
!python scripts/evaluate.py --n_step_each 5 --num_batches_to_samples 5 --batch_size 5 --model_path "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02" --tasks opt gen recon

/usr/local/lib/python3.11/site-packages/torch_geometric/typing.py:126: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/site-packages/torch_geometric/typing.py:164: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
Traceback (most recent call last):
  File "/content/cdvae/scripts/evaluate.py", line 11, in <module>
    from eval_utils import load_model
  File "/content/cdvae/scripts/eval_utils.py", line 16, in <module>
    from cdvae.common.data_utils import StandardScaler, chem

In [20]:
!pip install matminer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.6 MB/s eta 0:00:00


In [21]:
%%time
!python scripts/compute_metrics.py --root_path  "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02" --tasks   gen recon

Traceback (most recent call last):
  File "/content/cdvae/scripts/compute_metrics.py", line 16, in <module>
    from matminer.featurizers.site.fingerprint import CrystalNNFingerprint
  File "/usr/local/lib/python3.11/site-packages/matminer/featurizers/site/__init__.py", line 13, in <module>
    from matminer.featurizers.site.fingerprint import (
  File "/usr/local/lib/python3.11/site-packages/matminer/featurizers/site/fingerprint.py", line 11, in <module>
    from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import (
  File "/usr/local/lib/python3.11/site-packages/pymatgen/analysis/chemenv/coordination_environments/chemenv_strategies.py", line 18, in <module>
    from pymatgen.analysis.chemenv.coordination_environments.voronoi import DetailedVoronoiContainer
  File "/usr/local/lib/python3.11/site-packages/pymatgen/analysis/chemenv/coordination_environments/voronoi.py", line 9, in <module>
    import matplotlib.pyplot as plt
  File "/usr/local/lib/python3.11/si

In [22]:
temp_dir[0]

'2025-05-21'

In [23]:
!ls /content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02/

run.log


In [24]:
import torch
from jarvis.core.atoms import Atoms
from jarvis.core.atoms import pmg_to_atoms
from jarvis.core.lattice import Lattice
from jarvis.core.specie import atomic_numbers_to_symbols
from jarvis.db.jsonutils import dumpjson
from jarvis.analysis.structure.spacegroup import Spacegroup3D
from collections import Counter
from pymatgen.core.structure import Structure
import pandas as pd
opt_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_opt.pt"
gen_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_gen.pt"
recon_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_recon.pt"
csv_path = "/content/cdvae/data/supercon/test.csv"
df = pd.read_csv(csv_path)
x = torch.load(recon_path)
y = torch.load(gen_path)
z = torch.load(opt_path)
print(len(df),x["num_atoms"].shape,y["num_atoms"].shape,z["num_atoms"].shape)

FileNotFoundError: [Errno 2] No such file or directory: '/content/cdvae/HYDRA_JOBS/singlerun/2025-05-21/supercon_test02/eval_recon.pt'

In [ ]:
num_atoms = x["num_atoms"]
atom_types = x["atom_types"]
frac_coords = x["frac_coords"]
lengths = x["lengths"]
angles = x["angles"]
index_list = torch.cumsum(num_atoms[0], dim=0).numpy().tolist()
indice_tuples = []
for i, ii in enumerate(index_list):
    if i == 0:
        tup = [0, index_list[i] - 1]
    else:
        tup = [index_list[i - 1] - 1, index_list[i] - 1]
    indice_tuples.append(tup)

recon_structures = []

for id_needed in range(num_atoms.shape[1]):
    id_fracs = frac_coords[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_atom_types = atom_types[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_lengths = lengths[0].numpy()[id_needed]
    id_angles = angles[0].numpy()[id_needed]
    lat = Lattice.from_parameters(
        id_lengths[0],
        id_lengths[1],
        id_lengths[2],
        id_angles[0],
        id_angles[1],
        id_angles[2],
    ).matrix
    atoms = Atoms(
        lattice_mat=lat,
        elements=atomic_numbers_to_symbols(id_atom_types),
        coords=id_fracs,
        cartesian=False,
    )
    # spg_numb = Spacegroup3D(atoms).space_group_number
    # spg_numbs.append(spg_numb)

    # print()
    # print()
    # print()
    # print("jarvis\n", atoms)
    # struct = Structure(
    #    lattice=Lat.from_parameters(
    #        id_lengths[0],
    #        id_lengths[1],
    #        id_lengths[2],
    #        id_angles[0],
    #        id_angles[1],
    #        id_angles[2],
    #    ),
    #    species=id_atom_types,
    #    coords=id_fracs,
    #    coords_are_cartesian=False,
    # )
    # atoms = pmg_to_atoms(struct)
    # print("pmg\n", atoms)
    # print()
    # print()
    # print()

    # gen_structures.append(atoms.to_dict())
    recon_structures.append(atoms)


In [ ]:
test_structures=[]
jids = []
for i,ii in df.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  test_structures.append(atoms)#.to_dict())
  jids.append(ii['material_id'])

In [ ]:
df

Uploading to JARVIS-Leaderboard.

In [ ]:
from jarvis.io.vasp.inputs import Poscar
import json
f=open('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv','w')
line='id,target,prediction\n'
f.write(line)
for i,j,k in zip(test_structures,recon_structures,jids):
  print(k,i.composition.reduced_formula,j.composition.reduced_formula)
  line = k+","+Poscar(i).to_string().replace('\n','\\n')+","+Poscar(j).to_string().replace('\n','\\n')+"\n"
  #line = k+","+json.dumps(i.to_dict())+","+json.dumps(j.to_dict())+"\n"
  f.write(line)
f.close()
#zip file before uploading to JARVIS-Leadrboard

In [ ]:
dfx = pd.read_csv('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv')

In [ ]:
import pandas as pd
info = {}
test_path = pd.read_csv("/content/cdvae/data/supercon/test.csv")
val_path = pd.read_csv("/content/cdvae/data/supercon/val.csv")
train_path = pd.read_csv("/content/cdvae/data/supercon/train.csv")
test={}
val={}
train={}

for i,ii in train_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  train[jid]=pos

for i,ii in val_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  val[jid]=pos

for i,ii in test_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  test[jid]=pos
info['train']=train
info['val']=val
info['test']=test

In [ ]:
from jarvis.db.jsonutils import dumpjson
dumpjson(data=info,filename='dft_3d_Tc_supercon.json')

In [ ]:
test_path

In [ ]:
!cp scripts/compute_metrics.py scripts/eval_utils.py .

In [ ]:
!ls

In [ ]:
import numpy as np
from pymatgen.analysis.structure_matcher import StructureMatcher
import pandas as pd
from jarvis.io.vasp.inputs import Poscar
from tqdm import tqdm
df=pd.read_csv('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv')

matcher = StructureMatcher(stol=0.5, angle_tol=10, ltol=0.3)
rms = []
for m, mm in tqdm(df.iterrows()):
    try:
        atoms_target = (
            Poscar.from_string(
                (mm["target"].replace("\\n", "\n"))
            ).atoms
        ).pymatgen_converter()
        atoms_pred = (
            Poscar.from_string(
                (mm["prediction"].replace("\\n", "\n"))
            ).atoms
        ).pymatgen_converter()
        # rms_dist = matcher.get_rms_dist(atoms_pred,atoms_target)
        rms_dist = matcher.get_rms_anonymous(atoms_pred, atoms_target)
        if rms_dist[0] is not None:
            rms.append(rms_dist[0])
    except Exception as exp:
        print("exp", exp)
        pass
rms = round(np.array(rms).mean(), 4)
print('rms', rms)

In [ ]:
!conda env export

In [ ]:
!pip install matminer==0.9.0

In [ ]:
!rm -rf /usr/local/lib/python3.10/dist-packages/pandas*
!pip uninstall pandas -y
!pip install pandas==1.5.3

In [ ]:
from p_tqdm import p_map
from compute_metrics import GenEval,get_crystal_array_list,Crystal, RecEval
crys_array_list, true_crystal_array_list = get_crystal_array_list(recon_path)
pred_crys = p_map(lambda x: Crystal(x), crys_array_list)
gt_crys = p_map(lambda x: Crystal(x), true_crystal_array_list)
rec_evaluator = RecEval(pred_crys, gt_crys)
recon_metrics = rec_evaluator.get_metrics()


crys_array_list, _ = get_crystal_array_list(gen_path)
gen_crys = p_map(lambda x: Crystal(x), crys_array_list)
gen_evaluator = GenEval(gen_crys, gt_crys, eval_model_name='carbon')
gen_metrics = gen_evaluator.get_metrics()
print(recon_metrics)

In [ ]:
print(recon_metrics)

In [ ]:
print(gen_metrics)

In [ ]:
pip install alignn

In [ ]:
from alignn.pretrained import get_multiple_predictions


In [ ]:
atoms

In [ ]:
atoms.write_poscar('POSCAR')

Quickly predict Tc and other properties

In [ ]:
!pretrained.py --model_name jv_supercon_tc_alignn --file_format poscar --file_path POSCAR

In [ ]:
!pretrained.py --model_name jv_supercon_debye_alignn --file_format poscar --file_path POSCAR

In [ ]:
!pretrained.py --model_name jv_supercon_edos_alignn --file_format poscar --file_path POSCAR

In [ ]:
!pretrained.py --model_name jv_supercon_a2F_alignn --file_format poscar --file_path POSCAR

In [ ]:
!pretrained.py --model_name jv_formation_energy_peratom_alignn --file_format poscar --file_path POSCAR

In [ ]:
!pip freeze